In [1]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz

     |████████████████████████████████| 3.0 MB 15.1 MB/s eta 0:00:01
     |████████████████████████████████| 12.9 MB 26.1 MB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 34.1 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 2.9 MB/s  eta 0:00:01
  Created wheel for scispacy: filename=scispacy-0.2.4-py3-none-any.whl size=35203 sha256=f601f405ac648b342058362a2f4c313ef0089413c5f794fc8766611eb8bbb1c7
  Stored in directory: /root/.cache/pip/wheels/80/01/69/37a2ab4f9b61773c187d83257ffb31365a5ad57e7779ae5e92
Successfully built scispacy
ERROR: allennlp 0.9.0 has requirement spacy<2.2,>=2.1.0, but you'll have spacy 2.2.3 which is incompatible.
  Attempting uninstall: botocore
    Found existing installation: botocore 1.15.13
    Uninstalling botocore-1.15.13:
      Successfully uninstalled botocore-1.15.13
  Attempting uninstall: rsa
    Found existing installation: rsa 4.0
    Uninstalling rsa-4.0:
      Successfully uninstalled rsa-4.0
     |█████████████████████

  Created wheel for en-core-sci-lg: filename=en_core_sci_lg-0.2.4-py3-none-any.whl size=501343161 sha256=ea35283bd23e592ba516454d848ad01ab990d125d3a40065c8b34db19c4d0051
  Stored in directory: /root/.cache/pip/wheels/43/12/d5/dd85b5deab7738797c4d5358672df3616dda39acf570a3ef96
Successfully built en-core-sci-lg


In [4]:
import scispacy 
import spacy
from spacy import displacy
import pandas as pd

nlp = spacy.load("en_core_sci_lg")

In [6]:
df = pd.read_csv("/kaggle/input/drugvisdata/DrugVisData.csv")
output = df

# Turn all our sentences into parsed spaCy documents
docs = [nlp(str(sentence)) for sentence in df["sentence"].to_list()]

In [7]:
for i in range(len(df.index)):
    negation_tokens_1 = [tok for tok in docs[i] if tok.dep_ == 'neg' and tok.text == str(df.iloc[i]["drug"])]
    negation_tokens_2 = [tok for tok in docs[i] if tok.dep_ == 'neg' and tok.head.pos_ == 'VERB']
    negation_tokens = negation_tokens_1 + negation_tokens_2
    negation_head_tokens = [token.head for token in negation_tokens]
    
    text = [token.text for token in negation_head_tokens]
    dep = [token.dep_ for token in negation_head_tokens]
    head_text = [token.head.text for token in negation_head_tokens]
    head_pos = [token.head.pos_ for token in negation_head_tokens]
    child = [token.children for token in negation_head_tokens]

    output.loc[i,'text'] = ','.join(text)
    output.loc[i,'dep'] = ','.join(dep)
    output.loc[i,'head_text'] = ','.join(head_text)
    output.loc[i,'head_pos'] = ','.join(head_pos)
    #output.loc[i,'child'] = ','.join(child)
    if output.loc[i,'text'] != '':
        output.loc[i,'Is_Negated'] = 1
    else:
        output.loc[i,'Is_Negated'] = 0

In [8]:
output.to_csv('DrugVisdata_Negation_Output_Parsed_D.csv',index=False)

TODO: Use neuralcoref to fix co-reference (Thrombin is a drug. It doesn't work. --> Thrombin is a drug. Thrombin doesn't work.

TODO: Add additional rules to dependency parse. (If token.dep_ head Verb phrase == "NEG" ... then negated)

#### Pros:
- quick and easy

#### Cons:
- words like "ineffective" aren't recognized as a negative token
- rules need to be manually designed
- can be complex to design rules in cases such as 
    (a) sentences with buts
    (b) comparison of drugs
    (c) when there is a negation word which is unrelated to the drug Eg. "The drug is recommended in cases where the patient is not suffering from a prior chronical disease"
    (d) negation is understood only with prior medical knowledge
- false positives due to double negation such as "not ineffective" or "not useless"